In [54]:
pip install google-adk


Note: you may need to restart the kernel to use updated packages.


In [55]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Setup and authentication complete.


In [56]:
import uuid
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool


In [57]:
retry_config = types.HttpRetryOptions(
    attempts=5,  
    exp_base=7,  
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], 
)

## mcp tool


In [58]:
mcp_image_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="npx",  
            args=[
                "-y", 
                "@modelcontextprotocol/server-everything",
            ],
            tool_filter=["getTinyImage"],
        ),
        timeout=30,
    )
)

print("✅ MCP Tool created")

✅ MCP Tool created


## ToolContext 

Single image request (1 image): Auto-approve, generate immediately

Bulk request (>1 image): Pause and ask for approval before generating multiple images

In [59]:
threshold = 1
def image_gen_request(
    requests: int, tool_context: ToolContext
) -> dict:
    """Requests for an image generation. Requires approval if requesting more than 1 time (threshold)."""
    
    
    if requests <= threshold:
        return {
            "status": "approved",
            "requests": requests,
            "message": f"auto-approved: {requests} ",
        }
    
    
    if not tool_context.tool_confirmation:
        tool_context.request_confirmation(
            hint=f"⚠️ Large requests: {requests}. Do you want to approve?",
            payload={"requests":requests},
        )
        return {  
            "status": "pending",
            "message": f"Requests for {requests} requires approval",
        }
   
    if tool_context.tool_confirmation.confirmed:
        return {
            "status": "approved",
            "requests": requests, 
            
            "message": f"approved: {requests}",
        }
        
  
    else:
        return {
            "status": "rejected",
            "message": f"rejected: {requests} ",
        }

## agent

In [60]:
image_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="image_agent",
    instruction="""Use the MCP Tool to generate images for user queries.
     when the user gives a request:
     1.use image_gen_request tool
     2. if the atatus is pending inform the user for its approval
     3. After receiving the final result, provide a clear summary""",
    tools=[mcp_image_server,FunctionTool(func=image_gen_request)],
    
)
print("agent created")

agent created


In [61]:
image_app = App(
    name="shipping_coordinator",
    root_agent=image_agent,
    resumability_config=ResumabilityConfig(is_resumable=True),
)

print("✅ Resumable app created!")

✅ Resumable app created!


/tmp/ipykernel_47/3981580449.py:4: UserWarning: [EXPERIMENTAL] ResumabilityConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  resumability_config=ResumabilityConfig(is_resumable=True),


In [62]:
session_service = InMemorySessionService()


image_runner = Runner(
    app=image_app, 
    session_service=session_service,
)

print("✅ Runner created!")


✅ Runner created!


#Helper function

In [63]:
def check_for_approval(events):
    """Check if events contain an approval request.

    Returns:
        dict with approval details or None
    """
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if (
                    part.function_call
                    and part.function_call.name == "adk_request_confirmation"
                ):
                    return {
                        "approval_id": part.function_call.id,
                        "invocation_id": event.invocation_id,
                    }
    return None

In [64]:
def print_agent_response(events):
    """Print agent's text responses from events."""
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if part.text:
                    print(f"Agent > {part.text}")

In [65]:
def create_approval_response(approval_info, approved):
    """Create approval response message."""
    confirmation_response = types.FunctionResponse(
        id=approval_info["approval_id"],
        name="adk_request_confirmation",
        response={"confirmed": approved},
    )
    return types.Content(
        role="user", parts=[types.Part(function_response=confirmation_response)]
    )


print(" Helper functions defined")

 Helper functions defined


In [ ]:
async def run_workflow(query: str, auto_approve: bool = True):
    """Runs a shipping workflow with approval handling.

    Args:
        query: User's image generation request
        auto_approve: Whether to auto-approve requests more than 1(simulates human decision)
    """

    print(f"\n{'='*60}")
    print(f"User > {query}\n")

    
    session_id = f"order_{uuid.uuid4().hex[:8]}"

    
    await session_service.create_session(
        app_name="shipping_coordinator", user_id="test_user", session_id=session_id
    )

    query_content = types.Content(role="user", parts=[types.Part(text=query)])
    events = []

   
    async for event in image_runner.run_async(
        user_id="test_user", session_id=session_id, new_message=query_content
    ):
        events.append(event)

    
    approval_info = check_for_approval(events)

    
    if approval_info:
        print(f"⏸️  Pausing for approval...")
        print(f"🤔 Human Decision: {'APPROVE ✅' if auto_approve else 'REJECT ❌'}\n")

       
        async for event in image_runner.run_async(
            user_id="test_user",
            session_id=session_id,
            new_message=create_approval_response(
                approval_info, auto_approve
            ), 
            invocation_id=approval_info[
                "invocation_id"
            ],  
        ):
            if event.content and event.content.parts:
                for part in event.content.parts:
                    if part.text:
                        print(f"Agent > {part.text}")

   
    else:
       
        print_agent_response(events)

    print(f"{'='*60}\n")


print("✅ Workflow function ready")

✅ Workflow function ready


In [67]:
await run_workflow("Generate 2 images")
await run_workflow("Generate 1 image")



User > Generate 2 images



/usr/local/lib/python3.11/dist-packages/google/adk/tools/mcp_tool/mcp_tool.py:101: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(
/usr/local/lib/python3.11/dist-packages/google/adk/tools/tool_context.py:92: UserWarning: [EXPERIMENTAL] ToolConfirmation: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  ToolConfirmation(
/usr/local/lib/python3.11/dist-packages/google/adk/agents/invocation_context.py:298: UserWarning: [EXPERIMENTAL] BaseAgentState: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  self.agent_states[event.author] = BaseAgentState()


⏸️  Pausing for approval...
🤔 Human Decision: APPROVE ✅



User > Generate 1 image



Agent > Image generation approved.
The image will be generated.



NameError: name 'response' is not defined